In [1]:
! pip install pyyaml h5py

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'], axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [5]:
# Look at APPLICATION_TYPE value counts for binning
app_Type_counts = application_df['APPLICATION_TYPE'].value_counts()
app_Type_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(app_Type_counts[app_Type_counts < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
class_counts = application_df['CLASSIFICATION'].value_counts()
class_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(class_counts[class_counts < 1000].index)
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_dummies  = pd.get_dummies(application_df)

In [10]:
# Split our preprocessed data into our features and target arrays
X = application_dummies.drop(['IS_SUCCESSFUL'], axis=1)
y = application_dummies['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=47)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
!pip install keras-tuner --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 1.7 MB/s eta 0:00:00


In [19]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# add extra hidden layer
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 10
hidden_nodes_layer2 = 8
hidden_nodes_layer3= 3

nn_model1 = tf.keras.models.Sequential()

# First hidden layer

nn_model1.add(tf.keras.layers.Dense(units=hidden_nodes_layer1,
             input_dim=number_input_features, activation="tanh"))

# Second hidden layer
nn_model1.add(tf.keras.layers.Dense(
    units=hidden_nodes_layer2, activation="relu"))

# third hidden layer
nn_model1.add(tf.keras.layers.Dense(
    units=hidden_nodes_layer3, activation="relu"))

# Output layer(sigmoid because it's binary classifier))
nn_model1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 10)                440       
                                                                 
 dense_4 (Dense)             (None, 8)                 88        
                                                                 
 dense_5 (Dense)             (None, 3)                 27        
                                                                 
 dense_6 (Dense)             (None, 1)                 4         
                                                                 
Total params: 559
Trainable params: 559
Non-trainable params: 0
_________________________________________________________________


In [20]:
# Compile the model
nn_model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [21]:
# Train the model
fit_model = nn_model1.fit(X_train_scaled, y_train, epochs=100)


Epoch 1/100
804/804 [==============================] - 5s 2ms/step - loss: 0.6007 - accuracy: 0.6833
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5629 - accuracy: 0.7230
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5590 - accuracy: 0.7261
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5558 - accuracy: 0.7282
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5541 - accuracy: 0.7299
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5533 - accuracy: 0.7295
Epoch 7/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5525 - accuracy: 0.7298
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5518 - accuracy: 0.7311
Epoch 9/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5515 - accuracy: 0.7306
Epoch 10/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5509 - accura

In [22]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5519 - accuracy: 0.7286 - 489ms/epoch - 2ms/step
Loss: 0.5518664121627808, Accuracy: 0.7286297082901001


In [23]:
# Export our model to HDF5 file  AlphabetSoupCharity
nn_model1.save("/Users/ritta/Documents/bootcamp/deep-learning-challenge/Starter_Code/HDF5/AlphabetSoupCharity_model1.h5")


Add more neurons to a hidden layer and changing the activation function

In [24]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# add extra hidden layer
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 40
hidden_nodes_layer2 = 10
hidden_nodes_layer3= 5

nn_model2 = tf.keras.models.Sequential()

# First hidden layer

nn_model2.add(tf.keras.layers.Dense(units=hidden_nodes_layer1,
             input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn_model2.add(tf.keras.layers.Dense(
    units=hidden_nodes_layer2, activation="sigmoid"))

# third hidden layer
nn_model1.add(tf.keras.layers.Dense(
    units=hidden_nodes_layer3, activation="sigmoid"))

# Output layer(sigmoid because it's binary classifier))
nn_model2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 40)                1760      
                                                                 
 dense_8 (Dense)             (None, 10)                410       
                                                                 
 dense_10 (Dense)            (None, 1)                 11        
                                                                 
Total params: 2,181
Trainable params: 2,181
Non-trainable params: 0
_________________________________________________________________


In [27]:
# Compile the model
nn_model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [28]:
# Train the model
fit_model = nn_model2.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5930 - accuracy: 0.7064
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5661 - accuracy: 0.7278
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5602 - accuracy: 0.7285
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5567 - accuracy: 0.7303
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5547 - accuracy: 0.7300
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5533 - accuracy: 0.7301
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5519 - accuracy: 0.7307
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5508 - accuracy: 0.7315
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5503 - accuracy: 0.7315
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5495 - accura

In [29]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5506 - accuracy: 0.7304 - 487ms/epoch - 2ms/step
Loss: 0.5506306886672974, Accuracy: 0.7303789854049683


In [30]:
# Export our model to HDF5 file  AlphabetSoupCharity
nn_model2.save("/Users/ritta/Documents/bootcamp/deep-learning-challenge/Starter_Code/HDF5/AlphabetSoupCharity_model2.h5")


since the accuracy is not getting better with changeing the number of hidden layers or changing the activation function, i will try tocreate bin for the name column.

In [31]:
#  Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [32]:
# Drop the  'EIN'
application_df = application_df.drop(columns = ['EIN'])


In [33]:
# find the count of unique values for the columns
for x in application_df.columns:
    print(x, len(application_df[x].unique()))

NAME 19568
APPLICATION_TYPE 17
AFFILIATION 6
CLASSIFICATION 71
USE_CASE 5
ORGANIZATION 4
STATUS 2
INCOME_AMT 9
SPECIAL_CONSIDERATIONS 2
ASK_AMT 8747
IS_SUCCESSFUL 2


In [34]:
# Look at NAMES value counts for binning
counts = application_df['NAME'].value_counts()
counts

PARENT BOOSTER USA INC                                                  1260
TOPS CLUB INC                                                            765
UNITED STATES BOWLING CONGRESS INC                                       700
WASHINGTON STATE UNIVERSITY                                              492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                          408
                                                                        ... 
ST LOUIS SLAM WOMENS FOOTBALL                                              1
AIESEC ALUMNI IBEROAMERICA CORP                                            1
WEALLBLEEDRED ORG INC                                                      1
AMERICAN SOCIETY FOR STANDARDS IN MEDIUMSHIP & PSYCHICAL INVESTIGATI       1
WATERHOUSE CHARITABLE TR                                                   1
Name: NAME, Length: 19568, dtype: int64

In [36]:
# Choose a cutoff value and create a list of NAMES to be replaced

names_to_replace = list(counts[counts<10].index)

# Replace in dataframe
for app in names_to_replace:
    application_df['NAME'] = application_df['NAME'].replace(app,"Other")

# Check to make sure binning was successful
application_df['NAME'].value_counts()

Other                                      21022
PARENT BOOSTER USA INC                      1260
TOPS CLUB INC                                765
UNITED STATES BOWLING CONGRESS INC           700
WASHINGTON STATE UNIVERSITY                  492
                                           ...  
CASCADE 4-H FOUNDATION                        10
FREE & ACCEPTED MASONS OF WASHINGTON          10
NEW MEXICO GARDEN CLUBS INC                   10
NATIONAL ASSOCIATION OF HISPANIC NURSES       10
UNION OF CALIFORNIA STATE WORKERS             10
Name: NAME, Length: 223, dtype: int64

In [37]:
# Look at CLASSIFICATION value counts for binning
classification_bin = application_df['CLASSIFICATION'].value_counts()
classification_bin.head()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [39]:
# You may find it helpful to look at CLASSIFICATION value counts >1
counts_classification = classification_bin[classification_bin>1]
counts_classification

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [40]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_bin[classification_bin<100].index)
classifications_to_replace

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64

In [41]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df,dtype=float)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,NAME_ACE MENTOR PROGRAM OF AMERICA INC,NAME_AIR FORCE ASSOCIATION,NAME_ALABAMA FEDERATION OF WOMENS CLUBS,NAME_ALABAMA TREASURE FOREST ASSOCIATION,NAME_ALPHA PHI SIGMA,NAME_ALPHA PHI SIGMA INC,NAME_ALTRUSA INTERNATIONAL FOUNDATION INC,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [42]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop('IS_SUCCESSFUL', axis=1).values
y = application_df['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 47)



In [45]:
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 7
hidden_nodes_layer2 = 14
hidden_nodes_layer3 = 21

nn_model3 = tf.keras.models.Sequential()

# First hidden layer

nn_model3.add(tf.keras.layers.Dense(units=hidden_nodes_layer1,
             input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn_model3.add(tf.keras.layers.Dense(
    units=hidden_nodes_layer2, activation="relu"))

# third hidden layer
nn_model3.add(tf.keras.layers.Dense(
    units=hidden_nodes_layer3, activation="relu"))

# Output layer(sigmoid because it's binary classifier))
nn_model3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model3.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 7)                 308       
                                                                 
 dense_19 (Dense)            (None, 14)                112       
                                                                 
 dense_20 (Dense)            (None, 21)                315       
                                                                 
 dense_21 (Dense)            (None, 1)                 22        
                                                                 
Total params: 757
Trainable params: 757
Non-trainable params: 0
_________________________________________________________________


In [46]:
# Compile the model
nn_model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [47]:
# Train the model
fit_model = nn_model3.fit(X_train_scaled,y_train,validation_split=0.15, epochs=50)

Epoch 1/50
684/684 [==============================] - 4s 3ms/step - loss: 0.6115 - accuracy: 0.6789 - val_loss: 0.5759 - val_accuracy: 0.7178
Epoch 2/50
684/684 [==============================] - 2s 3ms/step - loss: 0.5687 - accuracy: 0.7223 - val_loss: 0.5628 - val_accuracy: 0.7264
Epoch 3/50
684/684 [==============================] - 2s 3ms/step - loss: 0.5623 - accuracy: 0.7259 - val_loss: 0.5591 - val_accuracy: 0.7284
Epoch 4/50
684/684 [==============================] - 2s 2ms/step - loss: 0.5584 - accuracy: 0.7260 - val_loss: 0.5589 - val_accuracy: 0.7279
Epoch 5/50
684/684 [==============================] - 2s 2ms/step - loss: 0.5564 - accuracy: 0.7286 - val_loss: 0.5570 - val_accuracy: 0.7302
Epoch 6/50
684/684 [==============================] - 2s 2ms/step - loss: 0.5552 - accuracy: 0.7287 - val_loss: 0.5550 - val_accuracy: 0.7289
Epoch 7/50
684/684 [==============================] - 2s 2ms/step - loss: 0.5539 - accuracy: 0.7291 - val_loss: 0.5546 - val_accuracy: 0.7323
Epoch 

In [48]:
#Evaluate the model using the test data
model_loss, model_accuracy = nn_model3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5519 - accuracy: 0.7319 - 471ms/epoch - 2ms/step
Loss: 0.5518795847892761, Accuracy: 0.7318950295448303


In [50]:
# Export our model to HDF5 file  AlphabetSoupCharity
nn_model3.save("/Users/ritta/Documents/bootcamp/deep-learning-challenge/Starter_Code/HDF5/AlphabetSoupCharity_model3.h5")
